# Тема курсовой: "Болталка", "Бот-всезнайка"

##"Болталка"

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.3 MB/s 
     |████████████████████████████████| 182 kB 66.4 MB/s 
     |████████████████████████████████| 7.6 MB 44.5 MB/s 


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 7.1 MB/s 
     |████████████████████████████████| 132 kB 62.0 MB/s 
     |████████████████████████████████| 212 kB 66.3 MB/s 
     |████████████████████████████████| 127 kB 62.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_dataset

dataset = load_dataset("radm/tathagata")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/radm___text/radm--tathagata-6b19756d75eb994b/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset['train'][:5]

{'text': ['Всё, наблюдаемое в мире, лишено усилий и активности, ибо всё сущее в мире подобно сну или чудесно возникшим образам. Этого не понимают философы и невежды. Но те, кто видит сущее именно таким, видит его истинно. Те же, кто воспринимает его иначе, вовлечены в различения и, будучи зависимы от них, привязаны к двойственности. Восприятие мира с позиции различения подобно изучению собственного отражения в зеркале, или собственной тени, или луны, отражённой в воде, или эха, услышанного в долине. Люди, опутанные тенями собственного различения, привязываются к той или иной вещи и, оказываясь не в состоянии избавиться от двойственности, продолжают беспрестанное различение и потому никогда не достигают успокоения. Под успокоением понимается Единость, а Единостью порождается наивысшее Самадхи, обретаемое вступлением в царство Благородной Мудрости, выявляемой лишь в глубочайших недрах сознания.',
  'Поскольку невежды и простаки, не ведающие, что мир является лишь чем-то самим умом вообра

### Возьмём модель предобученную на этом датасэте

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

model_name_or_path = "radm/rugpt3medium-tathagata"
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)


In [ ]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=end_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [ ]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    if next_sentence == 'пока': break
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
В чем смысл жизни?
Пока ты размышляешь, что есть добро и что есть зло, наступит день, когда ты перестанешь искать добро и перестанешь искать зло.
Умно
Нет ума — нет мудрости. Мудрость приходит вместе с знанием. Знание приходит вместе с освобождением
ну ну
Что плохого в том, чтобы просто сидеть и ждать? Что плохого в том, чтобы просто хотеть жить? Всё является следствием несознаваемых влечений
привет
ты не любишь меня
почему
ты никогда не был со мной
вчера
это было давно
пока


## "Бот-всезнайка"

In [ ]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=37b0cdbfa1d5fbc70cddd9842fbfc485d42a761d91c7a8704c164e73aac7904f
  Stored in directory: /root/.cache/pip/wheels/07/93/05/72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia


In [ ]:
import wikipedia

In [ ]:
wikipedia.set_lang("ru")
wikipedia.page('Россия').content[:1000]

'Росси́я, или Росси́йская Федера́ция (сокр. РФ), — государство в Восточной Европе и Северной Азии. Россия — крупнейшее государство в мире, её территория в конституционных границах с территорией Крыма, аннексия которого не получила международного признания, составляет 17 125 191 км². Население страны в тех же границах составляет 147 182 123 чел. (2021; 9-е место в мире).\nСтолица — Москва. Государственный язык на всей территории страны — русский, в ряде регионов России также установлены свои государственные и официальные языки. Денежная единица — российский рубль.\nРоссия — многонациональное государство с широким этнокультурным многообразием. Согласно результатам переписей населения России 2010 года, а также Крыма и Севастополя 2014 года в стране живут представители свыше 190 национальностей, среди которых русские составляют свыше 80 %, а русским языком владеют свыше 99,4 % россиян. Бо́льшая часть населения (около 75 %) в религиозном отношении относит себя к православию, что делает Росс